In [75]:
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [76]:
csv_file_path = "covid/cough_trial_extended.csv"  
csv_data = pd.read_csv(csv_file_path)
audio_dir = "covid/trial_covid/" 

In [97]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    feature_list = []
    feature_list.append(np.mean(chroma_stft))
    feature_list.append(np.mean(rmse))
    feature_list.append(np.mean(spec_cent))
    feature_list.append(np.mean(spec_bw))
    feature_list.append(np.mean(rolloff))
    feature_list.append(np.mean(zcr))
    for e in mfcc:
        feature_list.append(np.mean(e))

    return feature_list[:19]

In [78]:
X = []
y = []
for index, row in csv_data.iterrows():
    file_name = os.path.join(audio_dir, row['file_properties'])
    features = extract_features(file_name)
    X.append(features)
    y.append(row['class'])

C:\Users\Sneha\anaconda3\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [79]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=40)

In [101]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

his=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test))
# Save the model
model.save("covid.h5")
model = load_model('covid.h5') 

Epoch 1/100
2/2 [==============================] - 1s 224ms/step - loss: 0.6686 - accuracy: 0.6471 - val_loss: 0.4175 - val_accuracy: 0.9143
Epoch 2/100
2/2 [==============================] - 0s 59ms/step - loss: 0.4361 - accuracy: 0.8824 - val_loss: 0.3056 - val_accuracy: 0.9143
Epoch 3/100
2/2 [==============================] - 0s 52ms/step - loss: 0.3244 - accuracy: 0.8824 - val_loss: 0.2357 - val_accuracy: 0.9143
Epoch 4/100
2/2 [==============================] - 0s 65ms/step - loss: 0.2649 - accuracy: 0.8824 - val_loss: 0.1962 - val_accuracy: 0.9143
Epoch 5/100
2/2 [==============================] - 0s 52ms/step - loss: 0.2325 - accuracy: 0.8824 - val_loss: 0.1767 - val_accuracy: 0.9143
Epoch 6/100
2/2 [==============================] - 0s 64ms/step - loss: 0.2125 - accuracy: 0.8824 - val_loss: 0.1630 - val_accuracy: 0.9143
Epoch 7/100
2/2 [==============================] - 0s 55ms/step - loss: 0.1871 - accuracy: 0.8824 - val_loss: 0.1488 - val_accuracy: 0.9143
Epoch 8/100
2/2 [==

Epoch 59/100
2/2 [==============================] - 0s 48ms/step - loss: 5.1741e-04 - accuracy: 1.0000 - val_loss: 0.1185 - val_accuracy: 0.9429
Epoch 60/100
2/2 [==============================] - 0s 57ms/step - loss: 4.5662e-04 - accuracy: 1.0000 - val_loss: 0.1214 - val_accuracy: 0.9429
Epoch 61/100
2/2 [==============================] - 0s 48ms/step - loss: 3.8822e-04 - accuracy: 1.0000 - val_loss: 0.1244 - val_accuracy: 0.9429
Epoch 62/100
2/2 [==============================] - 0s 69ms/step - loss: 3.6225e-04 - accuracy: 1.0000 - val_loss: 0.1274 - val_accuracy: 0.9429
Epoch 63/100
2/2 [==============================] - 0s 87ms/step - loss: 3.3837e-04 - accuracy: 1.0000 - val_loss: 0.1301 - val_accuracy: 0.9429
Epoch 64/100
2/2 [==============================] - 0s 73ms/step - loss: 3.2851e-04 - accuracy: 1.0000 - val_loss: 0.1326 - val_accuracy: 0.9429
Epoch 65/100
2/2 [==============================] - 0s 61ms/step - loss: 3.1470e-04 - accuracy: 1.0000 - val_loss: 0.1347 - val_ac

C:\Users\Sneha\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [109]:
_, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {test_accuracy:.4f}")

2/2 [==============================] - 0s 7ms/step - loss: 0.1663 - accuracy: 0.9429
Test Accuracy: 0.9429


In [98]:
def preprocess_features(features):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(np.array(features).reshape(1, -1))
    return features_scaled


In [99]:
model = load_model('covid.h5')  
audio_file_path = 'C:/Users/Sneha/Desktop/Test/Test_Data-20240412T131115Z-001/Test_Data/kOshish_mru.wav'

tf = extract_features(audio_file_path)

tfs = preprocess_features(tf)

test = model.predict(tfs)

if test[0] >= 0.86:
    print("Predicted class: COVID-19")
else:
    print("Predicted class: Non-COVID-19")


C:\Users\Sneha\anaconda3\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


1/1 [==============================] - 0s 75ms/step
Predicted class: Non-COVID-19


In [108]:
model = load_model('covid.h5')  
audio_file_path = 'C:/Users/Sneha/Desktop/Test/Test_Data-20240412T131115Z-001/Test_Data/pos1.wav'

tf = extract_features(audio_file_path)

tfs = preprocess_features(tf)

test = model.predict(tfs)

if test[0] >= 0.86:
    print("Predicted class: COVID-19")
else:
    print("Predicted class: Non-COVID-19")


1/1 [==============================] - 0s 78ms/step
Predicted class: COVID-19
